In [47]:
import elasticsearch
from elasticsearch_dsl import Search
import random
import time
import collections
from math import log
from sklearn.linear_model import LinearRegression
# from sklearn.linear_model import LogisticRegression


In [2]:
query_arr = []
with open("query_desc.51-100.short.txt") as file:
    text = file.read().split("\n")[:-1]
    for line in text:
        line = line.split(".   ")
        query_arr.append(line[0])

In [3]:
def filetodict_qrels(file_path):
    qrel_file = {}
    with open(file_path) as file:
        text = file.read().split("\n")[:-1]
    for line in text:
        line = line.split()
        if line[0] not in qrel_file:
            qrel_file[line[0]] = {}
        _dict = {
            "AssessorID" : line[1],
            "DocID" : line[2],
            "Grade" : line[3]
        }
        qrel_file[line[0]][line[2]] = _dict
    return qrel_file
    

In [4]:
desc = filetodict_qrels("qrels.adhoc.51-100.AP89.txt")

In [5]:
# doc = 0 
# unique_doc = {}
# for q in query_arr:
#     length = 0
#     for doc in desc[q]:
#         length += int(desc[q][doc]['Grade'])
#     print(q, length, len(desc[q]))
        
# #     print(q,len(desc[q]))

# # if q in query_arr:
# #         print(query desc[q])
# #     doc = doc + len(desc[q])
# # print(doc)

In [6]:
es = elasticsearch.Elasticsearch()

In [7]:
results = elasticsearch.helpers.scan(es,
    index="apdata",
    doc_type="doc",
    preserve_order=True,
    query={"query": {"match_all": {}}},
)
ids = []
url_ids = []
doc_no = {}

i = 0
id_doc_no_mapping = {}
doc_no_id_mapping = {}
for item in results:
    ids.append(item['_id'])
    url_ids.append(item['_source']['DOCNO'])
    doc_no[item['_source']['DOCNO']] = True
    
    id_doc_no_mapping[item['_id']] = item['_source']['DOCNO']
    doc_no_id_mapping[item['_source']['DOCNO']] = item['_id']
    


In [8]:
ids = list(set(ids))

In [9]:
doc = 0 
query_doc = {}
for q in query_arr:
    query_doc[q] = {}
    non_rev = 0
    uni = 0
    for doc in desc[q]:
        if int(desc[q][doc]['Grade']) == 1:
            query_doc[q][doc] = True
            uni = uni + 1
        else:
            query_doc[q][doc] = False
            non_rev = non_rev + 1
        
        cond = True
        
    while cond and non_rev <= 1000:
        sample_unique = random.sample(range(0, len(url_ids)), 2000)
        for index in sample_unique:
            if ids[index] not in query_doc[q]:
                query_doc[q][url_ids[index]] = False
                non_rev = non_rev + 1
            if non_rev >= 1000:
                cond = False
                break


In [10]:
x = 0
start = time.time()
vocab = {}
vocab_index = 0
reverse_vocabulary = []
sparse_vectorspace = {}
doc_word_count = {}
doc_length = {}
total_doc_length = 0
total_term_freq = {}
empty_doc = []
for _id in ids:
    x = x + 1
    response = es.termvectors(index = 'apdata', 
                   doc_type = 'doc', 
                   id = _id, 
                   fields = ["TEXT"], 
                   payloads=False, 
                   positions=False,
                   offsets=False,
                   term_statistics=True,
                   field_statistics=True)
    sparse_doc = {}
    word_count = 0
    if(response['term_vectors'] != {}):
        terms = response['term_vectors']['TEXT']['terms']
        for term in terms.keys():
            if term not in vocab.keys():
                vocab[term] = vocab_index
                vocab_index = vocab_index + 1
                reverse_vocabulary.append(term)
            doc_word_count[vocab[term]] = terms[term]['doc_freq']
            total_term_freq[vocab[term]] = terms[term]['ttf']
            sparse_doc[vocab[term]] = terms[term]['term_freq']
            word_count = word_count + terms[term]['term_freq']
    else:
        empty_doc.append(_id)
    sparse_vectorspace[_id] = sparse_doc
    doc_length[_id] = word_count
    total_doc_length = total_doc_length + word_count

    end = time.time()
print(end - start)


295.59491872787476


In [11]:
reverse_vocabulary[15558]

'rote'

In [13]:
# sparse_vectorspace['36709'][15558]

In [14]:
okaptf = {}
for vector in sparse_vectorspace.keys():
    terms = sparse_vectorspace[vector]
    doc = {}
    for term in terms.keys():
        doc[term] = (float(terms[term]) / float(terms[term] + 0.5 + (1.5 * float(doc_length[vector] / (total_doc_length / len(doc_length))   ))))
    okaptf[vector] = doc

In [15]:
queriesFile = '/home/parth/information_retrieval/Data/AP89_DATA/AP_DATA/query_desc.51-100.short.txt' 
f = open(queriesFile,"r", encoding='iso-8859-1')
Querytext = f.read()

queries = Querytext.split('\n')[0:-1]
query_dict = {}

for query in queries:
    temp = query.split("   ")
    query_dict[temp[0].split(".")[0]] = temp[1] 
analyse_body = {
  "analyzer": "my_english",
  "text": ""
}

for queryId in query_dict:
    analyse_body["text"] = query_dict[queryId]
    tokens = es.indices.analyze(index = "apdata", body = analyse_body)['tokens']
    query_dict[queryId] = [a['token'] for a in tokens if a['token'] != 'document']
    

In [17]:
okaptf_doc_scores = {}
for doc in query_dict:
    scores = {}
#     print(query_dict[doc])

    for vector in okaptf.keys():
        score = 0
        for word in query_dict[doc]:
            key_dict = okaptf[vector].keys()
            if(vocab[word] in key_dict):
                score = score + okaptf[vector][vocab[word]]
        scores[vector] = score
    okaptf_doc_scores[doc] = scores
    

In [18]:
for query in okaptf_doc_scores:
    okaptf_doc_scores[query] = sorted(okaptf_doc_scores[query].items(), key=lambda kv: kv[1])[::-1]

In [19]:
tfidf_doc_scores = {}
for doc in query_dict:
    scores = {}
    for vector in okaptf.keys():
        score = 0
        for word in query_dict[doc]:
            key_dict = okaptf[vector].keys()
            if(vocab[word] in key_dict):
                score = score + (okaptf[vector][vocab[word]] * (log(len(okaptf) / doc_word_count[vocab[word]]) ))
        scores[vector] = score
    tfidf_doc_scores[doc] = scores

In [20]:
for query in tfidf_doc_scores:
    tfidf_doc_scores[query] = sorted(tfidf_doc_scores[query].items(), key=lambda kv: kv[1])[::-1]

In [21]:
okapi_bm_doc_scores = {}
total_doc = len(ids)
k1 = 1.2
k2 = 1000
b = 0.75
avg_doc_len = total_doc_length / total_doc
avg_doc_len

247.80981116923914

In [22]:

okapi_bm_doc_scores = {}
for doc in query_dict:
    scores = {}
    w_q_count = collections.Counter(query_dict[doc])
    for vector in okaptf.keys():
        score = 0
        key_dict = okaptf[vector].keys()
        terms = sparse_vectorspace[vector]

        for word in query_dict[doc]:
            if(vocab[word] in key_dict):
                w_id = vocab[word]
                log_term = log((total_doc + 0.5) / (doc_word_count[w_id] + 0.5))
                
                word_doc_term_nume = terms[w_id] * (k1 + 1 )
                word_doc_term_den = terms[w_id] + k1 * ((1-b) + b * (doc_length[vector] / avg_doc_len))
                
                word_query_term = w_q_count[word] * ( 1 + k2) / (w_q_count[word] + k2)
                score = score + (log_term * (word_doc_term_nume / word_doc_term_den) * word_query_term)
        scores[vector] = score
    okapi_bm_doc_scores[doc] = scores

In [23]:
for query in okapi_bm_doc_scores:
    okapi_bm_doc_scores[query] = sorted(okapi_bm_doc_scores[query].items(), key=lambda kv: kv[1])[::-1]

In [24]:
unig_lap = {}
for doc in query_dict:
    scores = {}
    for vector in sparse_vectorspace.keys():
        score = 0
        terms = sparse_vectorspace[vector]
        for word in query_dict[doc]:
            key_dict = terms.keys()
            w_id = vocab[word]
            if(vocab[word] in key_dict):
                score = score + log((terms[w_id] + 1) / (doc_length[vector] + len(vocab)))
            else:
                score = score + log( 1 / (doc_length[vector] + len(vocab)))
        scores[vector] = score
    unig_lap[doc] = scores

In [25]:
for query in unig_lap:
    unig_lap[query] = sorted(unig_lap[query].items(), key=lambda kv: kv[1])[::-1]

In [26]:
l = 0.75
unig_JM = {}
for doc in query_dict:
    scores = {}
    for vector in sparse_vectorspace.keys():
        score = 0
        terms = sparse_vectorspace[vector]
        for word in query_dict[doc]:
            key_dict = terms.keys()
            w_id = vocab[word]
            if(vocab[word] in key_dict):
                p_a  = l * (terms[w_id] /  doc_length[vector])
                p_b = (1 - l) * (total_term_freq[w_id] - terms[w_id]) / ( total_doc_length - doc_length[vector])
                score = score + log(p_a + p_b)
            else:
                p_b = (1 - l) * (total_term_freq[w_id]) / ( total_doc_length - doc_length[vector])
                score = score + log(p_b)
        scores[vector] = score
    unig_JM[doc] = scores


In [27]:
for query in unig_JM:
    unig_JM[query] = sorted(unig_JM[query].items(), key=lambda kv: kv[1])[::-1]

In [28]:
tfidf_doc_scores_dict = {}
for q in tfidf_doc_scores:
    tfidf_doc_scores_dict[q] = {}
    for scores in tfidf_doc_scores[q]:
        tfidf_doc_scores_dict[q][scores[0]] = scores[1]
        
unig_JM_dict = {}
for q in unig_JM:
    unig_JM_dict[q] = {}
    for scores in unig_JM[q]:
        unig_JM_dict[q][scores[0]] = scores[1]
        
unig_lap_dict = {}
for q in unig_lap:
    unig_lap_dict[q] = {}
    for scores in unig_lap[q]:
        unig_lap_dict[q][scores[0]] = scores[1]
        
okapi_bm_doc_scores_dict = {}
for q in okapi_bm_doc_scores:
    okapi_bm_doc_scores_dict[q] = {}
    for scores in okapi_bm_doc_scores[q]:
        okapi_bm_doc_scores_dict[q][scores[0]] = scores[1]
        
okaptf_doc_scores_dict = {}
for q in okaptf_doc_scores:
    okaptf_doc_scores_dict[q] = {}
    for scores in okaptf_doc_scores[q]:
        okaptf_doc_scores_dict[q][scores[0]] = scores[1]

In [29]:
len(tfidf_doc_scores_dict),len(unig_JM_dict),len(unig_lap_dict),len(okapi_bm_doc_scores_dict),len(okaptf_doc_scores_dict)

(25, 25, 25, 25, 25)

In [30]:
# id_doc_no_mapping
# doc_no_id_mapping
# unig_JM
# unig_lap
# okapi_bm_doc_scores
# okaptf_doc_scores
# tfidf_doc_scores
# id_doc_no_mapping
# doc_no_id_mapping


feature = {}
for q in query_doc:
    feature[q] = {}
    for doc in query_doc[q]:
        
        doc1 = doc
        if(doc[-1] != " "):
            doc = doc + " "
            
#         query_dict[q]


        sparse_vectorspace[doc_no_id_mapping[doc]]

        w_q_c = 0
        for word in query_dict[q]:
            if vocab[word] in sparse_vectorspace[doc_no_id_mapping[doc]]:
                w_q_c += 1
    
        feature[q][doc] = {
            "tfidf"      : tfidf_doc_scores_dict[q][doc_no_id_mapping[doc]],
            "unigram_JM" : unig_JM_dict[q][doc_no_id_mapping[doc]],
            "unigram_lap": unig_lap_dict[q][doc_no_id_mapping[doc]],
            "okapi_bm"   : okapi_bm_doc_scores_dict[q][doc_no_id_mapping[doc]],
            "okapi_tf"   : okaptf_doc_scores_dict[q][doc_no_id_mapping[doc]],
            "num_word"   : w_q_c,
            "label"      : query_doc[q][doc1]
        }

In [165]:
id_dict = {}
index = 0

id_dict_test = {}
index_test = 0


qu = []
data = []
label = []
data_test = []
label_test = []
for query in feature:
    qu.append(query)
    for doc in feature[query]:
        arr = [0,0,0,0,0,0]
        arr[0] = feature[query][doc]["tfidf"]
        arr[1] = feature[query][doc]["unigram_JM"]
        arr[2] = feature[query][doc]["unigram_lap"]
        arr[3] = feature[query][doc]["okapi_bm"]
        arr[4] = feature[query][doc]["okapi_tf"]
        arr[5] = feature[query][doc]["num_word"]
        
        l = 0
        if feature[query][doc]["label"] == True:
            l = 1
#         l = feature[query][doc]["label"]
#         if len(qu) <= 20:
        if int(query) not in [56, 57, 64, 71, 99]:
            id_dict[query + "_" + doc[:-1]] = index
            index += 1
            data.append(arr)
            label.append(l)
        else:
            id_dict_test[query + "_" + doc[:-1]] = index_test
            index_test += 1


            data_test.append(arr)
            label_test.append(l)

            

In [122]:
regression = LinearRegression()

In [65]:
# log_regression = LogisticRegression()

In [166]:
regression.fit(data, label)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [167]:
pred = regression.predict(data)
test_pred = regression.predict(data_test)

In [125]:
# test_pred

In [170]:
ml_scores = {}
for _id in id_dict:
    query_doc = _id.split("_")
    if query_doc[0] not in ml_scores:
        ml_scores[query_doc[0]] = {}
    ml_scores[query_doc[0]][query_doc[1]] = pred[id_dict[_id]]
    

In [171]:
with open("ml_scores_train", "w") as file:
    for query in ml_scores:
        print(query)
        sorted_doc = sorted(ml_scores[query], key=ml_scores[query].__getitem__, reverse=True) 
        r = 1
        for doc in sorted_doc[0:1000]:
            file.write(str(query) + " Q0 " + doc + " " + str(r) + " " + str(ml_scores[query][doc]) + " Exp\n")
            r = r + 1

54
60
68
91
93
100
89
62
95
97
63
94
58
85
80
98
61
77
59
87


In [172]:
ml_scores = {}
for _id in id_dict_test:
    query_doc = _id.split("_")
    if query_doc[0] not in ml_scores:
        ml_scores[query_doc[0]] = {}
    ml_scores[query_doc[0]][query_doc[1]] = test_pred[id_dict_test[_id]]
    

In [173]:
with open("ml_scores_test", "w") as file:
    for query in ml_scores:
        print(query)
        sorted_doc = sorted(ml_scores[query], key=ml_scores[query].__getitem__, reverse=True) 
        r = 1
        for doc in sorted_doc[0:1000]:
            file.write(str(query) + " Q0 " + doc + " " + str(r) + " " + str(ml_scores[query][doc]) + " Exp\n")
            r = r + 1

56
57
64
71
99


In [77]:
# sum(test_pred)

In [103]:
for _id_ in sorted(range(len(test_pred)), key=lambda k: test_pred[k])[::-1]:
    print(test_pred[_id_], label[_id_])

0.7194243732416155 1
0.708513096014622 1
0.7023359541050926 0
0.7022530322708135 1
0.7019005041248495 1
0.6757714929593089 1
0.6704944666316156 1
0.669826985255086 1
0.6642063830842218 1
0.6637035882180907 1
0.6634088490026777 1
0.6578721985172797 1
0.6538811320164509 0
0.6527212375613737 1
0.6520385852266489 1
0.6473082913381386 1
0.6417491586929263 1
0.6387332449297484 1
0.6378005586826092 1
0.636095469378818 1
0.6354510317063832 1
0.6345964310667812 1
0.6304819460637358 1
0.6298924926533374 0
0.628787461945644 0
0.6237336846182487 1
0.622091345193563 1
0.6180492343980065 1
0.6177664735617624 1
0.6165515276993854 1
0.6159557916494348 1
0.6159351161486019 1
0.6156739412544252 1
0.6151746900346824 1
0.6139498070295369 1
0.6030251014569412 1
0.6023975454007408 1
0.6000205025252807 1
0.5984629123284247 0
0.588173182835845 1
0.5847781675231425 0
0.5844309120005167 0
0.5824114467470172 1
0.5806057825927221 0
0.577591216873837 1
0.5774326462861821 1
0.5763902683497254 0
0.5755407204815635 1

0.21942096393167024 0
0.2193537607154897 0
0.21928792037345204 0
0.21926249939985723 0
0.2191095176624492 1
0.2189155974256974 0
0.2188533877596906 0
0.21883347089422966 0
0.21878462829376563 0
0.21876933446945998 0
0.21845736304016403 0
0.2184346035962376 0
0.21842801718241542 1
0.2183854948646852 0
0.21826901261790346 1
0.21821865704707946 1
0.21821755478871813 1
0.21796465429125955 0
0.21792569932509886 0
0.2178070642385846 0
0.21779302245331866 0
0.21778457494281003 0
0.2176788259972974 0
0.21766942231085085 0
0.21761466668839874 0
0.21754437481884403 1
0.21751479948647037 0
0.21742283475782784 1
0.217289352888829 0
0.21725980414901525 0
0.21709987794778077 0
0.21708144317975273 0
0.2168535695699985 0
0.2168470424463791 0
0.21671611918127304 0
0.21670361295820792 0
0.2166608086229527 1
0.2166277952342848 0
0.2165701427366439 1
0.21656545884824221 0
0.2165097499395601 0
0.21650812406354178 0
0.21618743412149502 0
0.2160802874274839 1
0.21606780669437492 0
0.21605033669911425 0
0.215

0.14330237317347794 0
0.14326123036197566 0
0.14325807557169304 0
0.1432194574768672 0
0.14321914611174402 0
0.14319379754701414 0
0.14313566067427907 0
0.14310338249521762 1
0.14306823422214018 1
0.14305544305580564 0
0.1430446389220997 1
0.14301059846974748 0
0.14291354377674212 0
0.1428869979383721 0
0.14282981581371113 0
0.1428160734971188 0
0.14272828647424463 0
0.14271361840860713 0
0.14269214441313344 0
0.14258546750717974 0
0.14254336102526322 0
0.14253864360425952 0
0.14253514127078423 0
0.14251592415878558 0
0.14240059943601865 0
0.14238635348620138 0
0.1423292967746452 0
0.14232324854702105 0
0.1422594482119842 0
0.14224598525990154 0
0.14219875752012465 0
0.14218993699817423 0
0.14214853273196687 0
0.1420667841846095 0
0.14205705782677455 0
0.14203656325057679 0
0.1420218413782818 0
0.14197745915961468 0
0.14189449924297315 0
0.1418555304754799 1
0.1418309008357773 0
0.14183062275508884 0
0.14172719900911912 0
0.14163178281352615 0
0.14160170612493816 0
0.14158892264557116 

0.07337428219298892 0
0.07336210679407734 0
0.07323732618319195 0
0.0732084742185039 0
0.0730823719169538 0
0.07307594156544603 0
0.07304909022381054 0
0.07298980628439611 0
0.07291446108919739 0
0.07290392353651945 0
0.07289191591865382 0
0.07283528455863353 0
0.07276026938289032 0
0.0727530497012405 0
0.0727091381085389 0
0.07270161955577983 0
0.07262400437584801 0
0.07262058963503884 0
0.07259852863136529 0
0.0725975208557437 0
0.07259469006527122 0
0.07258448472560972 0
0.07257514346497115 0
0.0725554231621241 0
0.0725414531278607 0
0.07252951368894126 0
0.07249762938710919 0
0.07243819113577915 0
0.07242310738213528 0
0.07241101467690747 0
0.07237323442220858 0
0.07233428838466999 0
0.072333029059722 0
0.07230997172149667 0
0.07229585185466764 0
0.07224249569774943 0
0.07222087961762613 0
0.07220999421965757 0
0.07217851907638764 0
0.07213589306962617 0
0.07212268928297341 0
0.07207428662131507 0
0.07207244496273685 0
0.07205458044197063 0
0.07201852283998667 0
0.07201487069820292

0.03703311246960287 0
0.03697108400481291 0
0.036961841071610484 0
0.0369514493574886 0
0.03691380799151605 0
0.03689845041666643 0
0.036895530957807156 0
0.03689407943721141 0
0.036889314939699054 0
0.0368755789745823 0
0.0368635893186992 0
0.03685356050302239 0
0.03684745908764563 0
0.03684316831594434 0
0.03684161744518507 0
0.03682411861782723 0
0.03678437711454368 0
0.03677484183778668 0
0.03674707362980107 0
0.03674706345274884 0
0.036731939829693035 0
0.03669999670819348 0
0.0366571703029052 0
0.03665303636236631 0
0.036652846498849805 0
0.03665017550908018 0
0.03664479813331065 0
0.036621043901824545 0
0.036588403520348325 0
0.036588249297763296 0
0.036583997408606565 0
0.03656880690204294 0
0.036532517994060536 0
0.036531397041858434 0
0.036492637844863875 0
0.036482635139311374 0
0.03645914614699194 0
0.03644947288436596 0
0.036443623480433125 0
0.03643075999992524 0
0.036420511677675106 0
0.03641732572233664 0
0.036393685877030325 0
0.036392471291926035 0
0.0363605978713375 

...

-0.006211909100367209 0
-0.006211909100367209 0
-0.0062119579963605745 0
-0.0062119579963605745 0
-0.006212006892625958 0
-0.006212006892625958 0
-0.006212055789163444 0
-0.006212055789163444 0
-0.006212055789163444 0
-0.006212055789163444 0
-0.006212202480408137 0
-0.006212251378033835 0
-0.006212300275931622 0
-0.006212300275931622 0
-0.006212349174101371 0
-0.006212349174101371 0
-0.006212349174101371 0
-0.006212398072543278 0
-0.006212398072543278 0
-0.0062124469712572725 0
-0.0062124469712572725 0
-0.0062124958702432995 0
-0.006212544769501442 0
-0.006212544769501442 0
-0.006212544769501442 0
-0.006212593669031659 0
-0.006212593669031659 0
-0.0062126425688339496 0
-0.0062126425688339496 0
-0.006212691468908384 0
-0.006212691468908384 0
-0.006212740369254878 0
-0.006212740369254878 0
-0.006212740369254878 0
-0.00621278926987346 0
-0.00621278926987346 0
-0.00621278926987346 0
-0.00621278926987346 0
-0.006212887071926958 0
-0.006212887071926958 0
-0.006212887071926958 0
-0.0062128870

-0.011482829022784323 0
-0.011494537927365427 0
-0.011506296294903372 0
-0.011506296294903372 0
-0.01151508965166436 0
-0.011530859155993743 0
-0.011572547508787216 0
-0.011579472087997665 0
-0.011597361870976118 0
-0.011619386184963852 0
-0.011619386184963852 0
-0.011649362161849176 0
-0.011650678193211937 0
-0.01167143036028711 0
-0.01169746275769138 0
-0.011703051360220394 0
-0.01171949635790024 0
-0.011724874298979192 0
-0.011730201924307633 0
-0.011730201924307633 0
-0.011739066535429936 0
-0.011740709781817342 0
-0.011745891737383059 0
-0.011753565234902084 0
-0.011761314793502696 0
-0.011780897223419316 0
-0.011790513692019881 0
-0.01179696543573535 0
-0.011797899457480991 0
-0.011800600638859286 0
-0.01180134572289681 0
-0.011807820455213815 0
-0.011809269163989493 0
-0.011810977047690803 0
-0.011813862283241987 0
-0.011813862283241987 0
-0.011825193500543889 0
-0.011827421495869593 0
-0.011839130392146437 0
-0.011839130392146437 0
-0.011840662406422835 0
-0.011853599520672255 

-0.014682699956264235 0
-0.014682743372990283 0
-0.014682743372990283 0
-0.014682743372990283 0
-0.01468283020716634 0
-0.014682873624616281 0
-0.014683047296829226 0
-0.014683047296829226 0
-0.014683134134383581 0
-0.014683264392525049 0
-0.014683307812388212 0
-0.014683394652838515 0
-0.01468348149425417 0
-0.014683568336635179 0
-0.014683655179981596 0
-0.014683655179981596 0
-0.014683698602016841 0
-0.014683742024293449 0
-0.014683959139296925 0
-0.014684002563021749 0
-0.014684002563021749 0
-0.014684002563021749 0
-0.01468404598698788 0
-0.01468404598698788 0
-0.014684089411195471 0
-0.014684132835644396 0
-0.014684132835644396 0
-0.01468421968526646 0
-0.014684263110439529 0
-0.014684393387407242 0
-0.014684393387407242 0
-0.014684436813545927 0
-0.014684480239926002 0
-0.014684567093410435 0
-0.014684697375447649 0
-0.014684697375447649 0
-0.014684697375447649 0
-0.014684740803276287 0
-0.014684871088210517 0
-0.0146849145170048 0
-0.014684957946040529 0
-0.014684957946040529 0

-0.016473135139940857 0
-0.01650129829093982 0
-0.01650699626629209 0
-0.016512615920991618 0
-0.016514212793453685 0
-0.016517564582156856 0
-0.01655781518604884 0
-0.016566483656093707 0
-0.016570825160081744 0
-0.016586161764368133 0
-0.01660856057404967 0
-0.01661422983850571 0
-0.01661966592750256 0
-0.016627704878400616 0
-0.016640977401124128 0
-0.016645464353985638 0
-0.01664789835392299 0
-0.016657256615100195 0
-0.016700333548851164 0
-0.016708146656018434 0
-0.016767530472109583 0
-0.016773517997115454 0
-0.01680972526413923 0
-0.016817026099489756 0
-0.01682317828879846 0
-0.016827985365096672 0
-0.016836221055562615 0
-0.01683905113963153 0
-0.016846420196348903 0
-0.016864864527000513 0
-0.016889377258708614 0
-0.016930313148384328 0
-0.01697500174769557 0
-0.0169761299859005 0
-0.016978805355996378 0
-0.016987052289374076 0
-0.01699191063582426 0
-0.0170077302011706 0
-0.01701447375930836 0
-0.01701502718012015 0
-0.01701729872995291 0
-0.017021658055489823 0
-0.01703383

-0.05292529074955379 0
-0.0529253721885034 0
-0.05292545362790573 0
-0.052925535067761036 0
-0.052925616508069144 0
-0.052925616508069144 0
-0.05292577939004361 0
-0.05292577939004361 0
-0.052925860831710074 0
-0.052925942273829456 0
-0.052926023716401614 0
-0.052926105159426605 0
-0.05292618660290437 0
-0.05292618660290437 0
-0.05292634949121879 0
-0.05292659382708658 0
-0.052926675273281674 0
-0.05292675671992957 0
-0.05292675671992957 0
-0.05292700106259062 0
-0.052927082511050126 0
-0.052927082511050126 0
-0.052927163959962685 0
-0.05292732685914616 0
-0.05292748976014144 0
-0.05292748976014144 0
-0.052927571211318435 0
-0.052927652662948486 0
-0.05292781556756717 0
-0.05292797847399782 0
-0.05292797847399782 0
-0.05292797847399782 0
-0.052928059927892616 0
-0.052928059927892616 0
-0.052928059927892616 0
-0.05292814138224027 0
-0.05292814138224027 0
-0.05292814138224027 0
-0.05292822283704093 0
-0.05292822283704093 0
-0.05292830429229472 0
-0.052928467204160966 0
-0.052928630117839

In [100]:
test_pred[7979], label[]

(0.7194243732416155, 1)

In [79]:
# sorted(test_pred)[::-1]